# M5のおさらい

- M5ではtreeの形成は普通のdecision treeと同じように行う（？）
    - これは作りたいモデルとはたぶん異なる。
    

# M5PP

- ふたつに分割するアルゴリズムを作る
    - これをrecursiveに適用すればcascading treeになるはず
- 分割にあたっては線形モデルのエラーを最小化するようにする
- ただし、ifelse型のスイッチは最適化困難なので、sigmoidかtanhを用いる。
- 以下のようなmodelを小さい$a$でまずtrainし、徐々に$a$を大きくしていくのがいいだろうか？
　　$$\left[\tanh\left(a(x+b)\right) -1\right]\times\text{回帰式}$$
  
- regime switch用のファクターごとにモデルを作って比較するのが安直ではあるが、もっとスマートな方法はあるか
    - ものすごくきついL1正則項を入れればいい気がする

In [1]:
import tensorflow as tf
import numpy as np

C:\Users\t\Anaconda3\envs\python35\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## random data の生成

In [2]:
Nsample = 10000

In [3]:
Nfactor = 10

In [4]:
trX = np.random.randn(Nsample , Nfactor)

In [5]:
Nregime  = 2

In [6]:
A = np.random.randn(Nfactor , Nregime)

In [7]:
trZ = np.dot(trX , A)

In [8]:
Nregime_factor = 5

In [9]:
trX2 = np.random.randn(Nsample, Nregime_factor)

In [10]:
A2 = np.random.randn(Nregime_factor , 1)

In [11]:
trZ2 = np.dot(trX2 , A2)

M5やM5Pに倣い、regime switcherとしては、候補とするファクターひとつのみが含まれる不等式をカスケード式に用いることを想定

従って


In [12]:
trY = np.array([z[0]  if  trZ2[i , 0] > 0 else z[1] for i , z in enumerate(trZ)])

In [13]:
ybias = np.mean(trY)

In [14]:
yscaler = np.std(trY)

## 標準化しない場合

In [44]:
trY = trY2

## 標準化する場合

In [15]:
trY2 = (trY - ybias ) / yscaler

In [ ]:
np.mean(trY2)

In [ ]:
np.std(trY2)

## modelの定義

In [45]:
def model(X, X2 , w1a , w1b, w2 , tanh_scaler): # this network is the same as the previous one except with an extra hidden layer + dropout
    z1a = tf.matmul(X , w1a)
    z1b = tf.matmul(X , w1b)
    z2 = tf.matmul(X2 , w2)
    h = tf.nn.tanh(tf.multiply(z2 , tanh_scaler))
    my_one = tf.constant(dtype="float" , value = 1 )
    weight1a = tf.subtract(my_one , h) 
    weight1b = tf.subtract(h , my_one)
    elementwise_mul1a = tf.multiply(weight1a , z1a)
    elementwise_mul1b = tf.multiply(weight1b , z1b)


    return elementwise_mul1a + elementwise_mul1b

In [46]:
tanh_scaler = tf.Variable(0.1 , dtype="float")

In [47]:
X = tf.placeholder("float", [None, Nfactor])
X2 = tf.placeholder("float", [None, Nregime_factor])

In [48]:
Y2 = tf.placeholder("float")

In [49]:
w1a = tf.Variable(tf.random_normal(shape=[Nfactor , 1] , stddev=0.1))

In [50]:
w1b = tf.Variable(tf.random_normal(shape=[Nfactor , 1] , stddev=0.1))

In [51]:
w2 = tf.Variable(tf.random_normal(shape=[Nregime_factor , 1] , stddev=0.1))

In [52]:
y_model = model(X , X2 , w1a , w1b , w2 , tanh_scaler)

以下の3つは同等？：

In [53]:
cost = tf.reduce_mean(tf.square(Y2 - y_model))

In [ ]:
cost = tf.losses.mean_squared_error(labels=Y2 , predictions=y_model)

In [ ]:
cost = tf.reduce_mean(tf.squared_difference(y_model, Y2))

In [54]:
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) # construct an optimizer to minimize cost and fit line to my data

sandbox check用：

## run

In [55]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

### batchを切った場合

In [56]:
batch_size = 100
n_epoch = 50

In [41]:
Nsample = trX.shape[0]

In [42]:
for i in range(n_epoch):
    for z in zip(range(0 , Nsample , batch_size) , range(batch_size , Nsample + 1 , batch_size)):
        sess.run(train_op , feed_dict={X:trX[z[0]:z[1]] , X2:trX2[z[0]:z[1]] , Y2 : trY2[z[0]:z[1]]})
        #print(z)
    run_cost = sess.run(cost , feed_dict={X:trX , X2:trX2 , Y2 : trY2})
    print(run_cost)
    
    #sess.run(train_operation , feed_dict={X:trX[z[0]:z[1]] , Y : trY[z[0]:z[1]]})
    #predicted_decoded_teY = sess.run(tf.argmax(py_x, 1) , feed_dict = {X : teX})
    #run_precision = np.mean(decoded_teY == predicted_decoded_teY)
    #print(i , run_precision)

0.940922
0.9409213
0.9409211
0.9409211
0.94092095
0.9409208
0.9409198
0.94092023
0.94092023
0.9409195
0.9409196
0.94091773
0.94091666
0.9409162
0.94091445
0.9409155
0.9409142
0.940915
0.9409151
0.94091576
0.9409151
0.94091433
0.9409139
0.94091266
0.9409107
0.94090897
0.9409083
0.9409081
0.9409064
0.94090813
0.9409069
0.9409048
0.9409044
0.94090307
0.9409029
0.94090194
0.9409003
0.9409003
0.94089895
0.94089776
0.9408961
0.9408954
0.9408936
0.94089246
0.94089186
0.9408898
0.94088846
0.9408875
0.94088614
0.94088626


tanh_scaler(が十分にちいさくなければ？)の微妙な変化はcostに影響を与えないだろう・・


### 一度に全部のtraining setを使った場合

In [ ]:
for i in range(100):
    sess.run(train_op , feed_dict={X:trX , X2:trX2 , Y2 : trY2})
    #print(sess.run(w1a)[0,0])
    print(sess.run(tanh_scaler))
    #print(sess.run(cost , feed_dict={X:trX , X2:trX2 , Y2 : trY2}))

9.999689
9.999545
9.999478
9.999448
9.9994335
9.999427
9.999424
9.999423
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422
9.999422


In [ ]:
sess.run(w1b)

# sandbox

In [ ]:
sess.run(tanh_scaler)

In [ ]:
def model(X, X2 , w1a , w1b, w2): # this network is the same as the previous one except with an extra hidden layer + dropout
    z1a = tf.matmul(X , w1a)
    z1b = tf.matmul(X , w1b)
    z2 = tf.matmul(X2 , w2)
    h = tf.nn.tanh(z2)
    my_one = tf.constant(dtype="float" , value = 1 )
    weight1a = my_one 
    #weight1b = tf.subtract(h , my_one)
    elementwise_mul1a = tf.multiply(weight1a , z1a)
    #elementwise_mul1b = tf.multiply(weight1b , z1b)


    return elementwise_mul1a# + elementwise_mul1b

In [ ]:
sess.run(w1a)

In [ ]:
sess.run(w1a)

In [ ]:
y = sess.run(py , feed_dict={X:trX , X2:trX2})

In [ ]:
trY.shape

In [ ]:
y.shape

In [ ]:
run_cost.shape

In [ ]:
cost = tf.add(tf.constant(dtype="float" , value = 100 ) , tf.reduce_mean(tf.square(Y - y_model)))

## pattern 1 : obsolete

In [ ]:
trY = 2 * trX + np.random.randn(*trX.shape) * 0.33 # create a y value which is approximately linear but with some random noise

In [ ]:
trY = [z[0] + z[1] if  trZ2[i , 0] > 0 else z[0] - z[1] for i , z in enumerate(trZ)]

In [ ]:
def model(X, X2 , w, w2): # this network is the same as the previous one except with an extra hidden layer + dropout
    z = tf.matmul(X , w)
    z2 = tf.matmul(X2 , w2)
    h = tf.nn.tanh(z2)
    my_one = tf.constant(dtype="float" , value = 1 )
    weight = tf.subtract(my_one , h) 
    #weight2 = tf.subtract()
    #r = tf.matmul(tf.subtract(my_one , h) , z)
    elementwise_mul = tf.multiply(weight , z)


    return elementwise_mul

In [ ]:
w = tf.Variable(tf.random_normal(shape=[Nfactor , Nregime] , stddev=0.1))

In [ ]:
py = model(X , X2 , w , w2)